<a href="https://colab.research.google.com/github/notmanan/Depression-Detection-Through-Multi-Modal-Data/blob/master/BiLSTM_WordLevel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import warnings
warnings.filterwarnings("ignore")

Mounted at /content/drive


MAKE DATASET

In [2]:
import nltk
nltk.download('stopwords')
import numpy as np
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
import gc
import math
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from smart_open import open
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from keras.layers import Dropout
from matplotlib import pyplot as plt

dev_location = "302_P"
test_location = "300_P"
train_location = "303_P"

devData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/devdata.csv',delimiter=',',encoding='utf-8'))[:, 0:4]
testData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/testdata.csv',delimiter=',',encoding='utf-8'))[:, 0:4]
trainData = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/traindata.csv',delimiter=',',encoding='utf-8'))[:, 0:4]

dataset = np.concatenate((devData, np.concatenate((testData, trainData))))


# print(dataset)
gc.collect()      
max_num_words = 17
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/MCA Dataset/GoogleNews-vectors-negative300.bin', binary=True)
stop_words = set(stopwords.words('english'))

def checkDataPointExistence(patientID, split):
  for i in split:
    if(patientID == i[0]):
      return True
  return False

def getData(patientID, location):
  # print("PatientID: " + str(int(patientID)))
  retData = [int(patientID)]
  textD = getTextData(patientID, location)
  textD = np.array(textD)
  audioD = getAudioData(patientID, location, textD)
  return textD,audioD

def getTextData(patientID, location):
  fileName = "/content/drive/My Drive/MCA Dataset/"+ location + "/" + str(int(patientID)) + "_TRANSCRIPT.csv"
  file = np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8', engine='python'))
  outputList = []

  for i in range(len(file)):
    sentence = file[i][3]
    words = str(sentence).split(" ")
    totalWordLength = 0
    for word in words: 
      totalWordLength += len(word)

    sentenceStart = file[i][0] 
    sentenceEnd = file[i][1]
    speaker = file[i][2]
    if speaker == 'Ellie':
      continue
    else:
      speaker = 1

    totalTime = sentenceEnd - sentenceStart
    timePerChar = totalTime/totalWordLength

    wordStart = sentenceStart
    for word in words:
      wordEnd = wordStart + (timePerChar * len(word))
      appender = [wordStart, wordEnd, speaker]
      vector = list(returnWordToVec(word))
      for v in vector:
        appender.append(v)
      # print(appender)
      # print(len(appender))
      outputList.append(appender)
      wordStart = wordEnd

  return outputList


def remove_StopWords(sentence):
    filtered_sentence = [] 
    for w in sentence: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence


def returnWordToVec(word):
  word = str(word)
  try:
    if word[0] == '<':
        word = word[1:]
    if word[-1] == '>':
        word = word[0:-1]
  except:
    dfdsd = 3
  if(word in model):
      return np.array(model[word])
  else:
    return np.zeros((300))

def audioDataHelper(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    X = np.array(X)
    return X
    
def getAudioData(patientID, location, textD):
  fileName = "/content/drive/My Drive/MCA Dataset/"+ location + "/" + str(int(patientID)) + "_COVAREP.csv"
  data = pd.read_csv(fileName,header = None)
  data = data.iloc[:,:].values
  data = audioDataHelper(data)
  # print("Audio Raw Data:" + str(data.shape))
  sentenceDatas = []
  for sentence in textD:
    sentenceStartime = sentence[0]
    sentenceEndTime = sentence[1]
    startIndex = math.floor(sentenceStartime/0.01)
    endIndex = math.ceil(sentenceEndTime/0.01)
    sentenceData = data[startIndex: endIndex]
    sentenceData = np.average(sentenceData, axis = 0)
    sentenceData = np.array(sentenceData.reshape(1, -1))
    sentenceDatas.append(sentenceData)
  
  sentenceDatas = np.array(sentenceDatas)
  sentenceDatas = np.reshape(sentenceDatas, (textD.shape[0],-1))
  # print("Audio Final Data:" + str(sentenceDatas.shape))

  return sentenceDatas

# Xtrain = []
Ytrain = []
# Xtest = []
Ytest = []


audio_train = []
text_train = []

audio_test = []
text_test = []


lengths = []
for datapoint in dataset:
  # print(datapoint[0])
  if(checkDataPointExistence(datapoint[0], devData)):

    # Data Point in Dev Set
    text,audio = getData(datapoint[0], dev_location)
    text = text[:,2:]
    audio_train.append(audio)
    text_train.append(text)
    Ytrain.append(datapoint[2])
    # print(data)
  if(checkDataPointExistence(datapoint[0], testData)):
    # Data Point in Test Set
    text,audio = getData(datapoint[0], test_location)
    text = text[:,2:]
    audio_test.append(audio)
    text_test.append(text)
    Ytest.append(datapoint[2])
  elif(checkDataPointExistence(datapoint[0], trainData)):
    # Data Point in Train Set
    text,audio = getData(datapoint[0], train_location)
    text = text[:,2:]
    audio_train.append(audio)
    text_train.append(text)
    Ytrain.append(datapoint[2])
  # lengths.append(data.shape[0])
model = []
gc.collect()

def refactor(arr, size):
  temp = arr[0:min(len(arr),size), :]
  if (len(temp) < size):
    temp = np.concatenate((temp, np.zeros(((size - len(temp)), arr.shape[1]))), axis = 0 )
  return temp

numberOfSentences = 1700

for i in range(len(audio_train)):
  # Xtrain[i] = refactor(Xtrain[i], numberOfSentences)
  audio_train[i] = refactor(audio_train[i], numberOfSentences)
  text_train[i] = refactor(text_train[i], numberOfSentences)
  # print(Xtrain[i].shape)

for i in range(len(audio_test)):
  audio_test[i] = refactor(audio_train[i], numberOfSentences)
  text_test[i] = refactor(text_train[i], numberOfSentences)
  # print(Xtest[i].shape)

audio_test = np.array(audio_test)
text_test = np.array(text_test)
# text_test = text_test[:,:,2:]

audio_train = np.array(audio_train)
text_train = np.array(text_train)
# dataset = []
gc.collect()

print(audio_test.shape,text_test.shape)
print(audio_train.shape,text_train.shape)

dataset = []
devData = []
trainData = []
testdata = []
gc.collect()

Ytrain = np.array(Ytrain)
Ytest = np.array(Ytest)

gc.collect()

import sklearn
from sklearn import preprocessing

def upsample(X_train,Y_train):
  X_train_0 = X_train[Y_train<10]
  X_train_1 = X_train[Y_train>=10]

  Y_train_1 = Y_train[Y_train>=10]
  # print(Y_train_1.shape)
  # print(X_train_1.shape)
  size = X_train_0.shape[0] - X_train_1.shape[0]
  X = []
  Y = []
  X_train = list(X_train)
  Y_train = list(Y_train)
  while(size>0):
    size -= 1
    index = np.random.randint(0,X_train_1.shape[0]-1)
    leave_index = np.random.randint(0,len(X_train)-1)
    X_add = X_train_1[index]
    X_leave = X_train[leave_index]

    Y_add = Y_train_1[index]
    Y_leave = Y_train[leave_index]

    X_train[leave_index] = X_add
    X_train.append(X_leave)

    Y_train[leave_index] = Y_add
    Y_train.append(Y_leave)


  X_train = np.array(X_train)
  Y_train = np.array(Y_train)
  return X_train,Y_train


audio_train = np.nan_to_num(audio_train)
text_train = np.nan_to_num(text_train)


audio_train, _ = upsample(audio_train,Ytrain)
text_train, Ytrain = upsample(text_train,Ytrain)

gc.collect()

for i in range(audio_train.shape[0]):
  audio_train[i] = sklearn.preprocessing.normalize(audio_train[i])
  text_train[i] = sklearn.preprocessing.normalize(text_train[i])

audio_test = np.nan_to_num(audio_test)
text_test = np.nan_to_num(text_test)

for i in range(audio_test.shape[0]):
  audio_test[i] = sklearn.preprocessing.normalize(audio_test[i])
  text_test[i] = sklearn.preprocessing.normalize(text_test[i])

gc.collect()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


(25, 1700, 74) (25, 1700, 301)
(52, 1700, 74) (52, 1700, 301)


0

BILSTM MODEL WITH GATING (WORD LEVEL)

In [3]:
#Word Level

#Text+Audio
import tensorflow as tf

from tensorflow import keras

from keras.layers import LSTM, Dense
from keras import layers

import numpy as np

class Highway(layers.Layer):

  def __init__(self):
    super(Highway, self).__init__()

  def build(self, input_shape):
    n_sentences = input_shape[1]
    n_features = input_shape[2]
    carry_bias = keras.initializers.Constant(value=-1.0)
    random_dist = keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

    carry_bias_2 = keras.initializers.Constant(value= 0.1)

    self.W_T = self.add_weight(shape=(n_features, n_features),initializer = random_dist,trainable=True)
    self.b_T = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias, trainable=True)
    # self.W_T = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1),trainable=True)
    # self.b_T = tf.Variable(tf.constant(-1.0, shape=[n_data_points, n_sentences, n_features]),trainable=True)
   
    self.W = self.add_weight(shape=( n_features, n_features),initializer = random_dist, trainable=True)
    self.b = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias_2, trainable=True)
    # self.W = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1), trainable=True)
    # self.b = tf.Variable(tf.constant(0.1, shape=[n_data_points, n_sentences, n_features]), trainable=True)  


  def call(self, inputs):
    x = inputs
    T = tf.sigmoid(tf.matmul(x, self.W_T) + self.b_T, name="transform_gate")
    H = tf.nn.relu(tf.matmul(x, self.W) + self.b, name="activation")
    C = tf.subtract(1.0, T, name="carry_gate")
    
    return tf.add(tf.multiply(H, T), tf.multiply(x, C), "y")


# Multiple Inputs
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Bidirectional
from keras.layers import Dense
from keras.layers import concatenate

# first input model
input1 = Input(shape=(1700,74), name = 'Audio_input')
highway1 = Highway()(input1)
highway5 = Highway()(highway1)
highway6 = Highway()(highway5)
dense1 = Dense(74)(highway6)

# second input model
# input2 = Input(shape=(1700,388), name = 'Video_input')
# highway2 = Highway()(input2)
# highway3 = Highway()(highway2)
# highway4 = Highway()(highway3)
# # dense8 = Dense(1000)(highway4)
# # dense9 = Dense(500)(dense8)
# dense7 = Dense(200)(highway4)
# dense2 = Dense(74)(dense7) 

input3 = Input(shape = (1700,301), name = 'Text_input')
# dense4 = Dense(1000)(input3)
# dense5 = Dense(500)(dense4)
dense6 = Dense(150)(input3)
dense3 = Dense(74)(dense6)
# merge input models
merge = concatenate([dense1,dense3], axis = 1)
# interpretation model
lstm = LSTM(128, dropout = 0.2, recurrent_dropout = 0.2)
bidire = Bidirectional(lstm)
output = Dense(1, activation='sigmoid')(bidire(merge))
model = Model(inputs=[input1, input3], outputs=output)
# summarize layers
print(model.summary())
# plot graph
plot_model(model)
optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001)
lf = tf.keras.losses.MeanSquaredLogarithmicError()
model.compile(optimizer=optimizer, loss=lf, metrics=[ keras.metrics.MeanSquaredError(), keras.metrics.AUC(),])

# inp = np.array([audio_train,video_train,text_train], dtype = object)

# filepath = '/content/drive/My Drive/MCA Dataset/checkFile.txt'

# model.fit([audio_train,text_train],Ytrain, validation_split = 0.2, callbacks = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss', min_delta=0, patience=15, verbose=0, mode='min',
#     baseline=None, restore_best_weights=True),epochs=30, batch_size = 4)




# def Losses(Y_pred, Y_test):
#   sum = 0
#   for i in range(len(Y_pred)):
#     m = abs(Y_pred[i]-Y_test[i])**2




# from sklearn.metrics import classification_report
# pred = model.predict([audio_test, text_test])
# pred2 = model.predict([audio_train,video_train,text_train])

# print("Accuracy = ", model.score([audio_test,video_test,text_test],Ytest))

# print(classification_report(Ytest,Thresholding(pred,0.5)))
# print("TRAINING ACCCCCC        " , classification_report(Ytrain,Thresholding(pred2,0.5)))
# print(classification_report(Ytest,Thresholding(pred,0.6)))
# print(classification_report(Ytest,Thresholding(pred,0.4)))
# print(classification_report(Ytest,Thresholding(pred,0.3)))
# print(classification_report(Ytest,Thresholding(pred,0.7)))




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Audio_input (InputLayer)       [(None, 1700, 74)]   0           []                               
                                                                                                  
 highway (Highway)              (None, 1700, 74)     262552      ['Audio_input[0][0]']            
                                                                                                  
 highway_1 (Highway)            (None, 1700, 74)     262552      ['highway[0][0]']                
                                                                                                  
 Text_input (InputLayer)        [(None, 1700, 301)]  0           []                               
                                                                                              

In [5]:
model.fit([audio_train,text_train],Ytrain, validation_split = 0.2, callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=15, verbose=0, mode='min',
    baseline=None, restore_best_weights=True),epochs=8, batch_size = 2)

Epoch 1/8
31/31 [==============================] - 981s 32s/step - loss: 2.1225 - mean_squared_error: 65.4468 - auc: 0.5404 - val_loss: 3.3459 - val_mean_squared_error: 142.7298 - val_auc: 0.5000
Epoch 2/8
31/31 [==============================] - 983s 32s/step - loss: 2.0326 - mean_squared_error: 64.5459 - auc: 0.5000 - val_loss: 3.3435 - val_mean_squared_error: 142.7001 - val_auc: 0.5000
Epoch 3/8
31/31 [==============================] - 970s 31s/step - loss: 2.0320 - mean_squared_error: 64.5388 - auc: 0.5000 - val_loss: 3.3432 - val_mean_squared_error: 142.6960 - val_auc: 0.5000
Epoch 4/8
31/31 [==============================] - 966s 31s/step - loss: 2.0318 - mean_squared_error: 64.5369 - auc: 0.5000 - val_loss: 3.3430 - val_mean_squared_error: 142.6936 - val_auc: 0.5000
Epoch 5/8
31/31 [==============================] - 978s 32s/step - loss: 2.0317 - mean_squared_error: 64.5357 - auc: 0.5000 - val_loss: 3.3429 - val_mean_squared_error: 142.6922 - val_auc: 0.5000
Epoch 6/8
31/31 [===

In [8]:
print("Loss = ", model.evaluate([audio_test,text_test],Ytest))

1/1 [==============================] - 3s 3s/step - loss: 2.2364 - mean_squared_error: 102.9217 - auc: 0.5000
Loss =  [2.236429452896118, 102.92167663574219, 0.5]
